# Creating and sharing public links to nested resources in CloudStor

It's easy to create a public link to a file or folder in the CloudStor interface. But what if you have lots of files in nested folders that you want to make publicly accessible? Yes, you can share the parent folder, but how do you create direct public links to files and folders *inside* the parent folder without generating new share links for every resource.

This notebook walks demonstrates how to create links to resources nested within a shared folder. The code cells run some simple tests to try and make sure that the example links go where they're supposed to.

In [18]:
import re

import requests

This is a public link to a shared folder that contains the OCRd text of digitised journals from Trove:

<https://cloudstor.aarnet.edu.au/plus/s/QOmnqpGQCNCSC2h>

You can create these links within the CloudStor web interface by clicking on the 'share' icon.

In [19]:
# Test link
response = requests.get("https://cloudstor.aarnet.edu.au/plus/s/QOmnqpGQCNCSC2h")
assert response.status_code == 200

## Link to a child folder

If we want to open a folder that is a child of the shared folder, we add a `path` parameter that points to the child folder, in this case `/angry-penguins-broadsheet-nla.obj-320790312`:

<https://cloudstor.aarnet.edu.au/plus/s/QOmnqpGQCNCSC2h?path=/angry-penguins-broadsheet-nla.obj-320790312>

In [3]:
# Test link
response = requests.get(
    "https://cloudstor.aarnet.edu.au/plus/s/QOmnqpGQCNCSC2h?path=/angry-penguins-broadsheet-nla.obj-320790312"
)
assert response.status_code == 200

## Download a child folder

The link above opens the child folder within the CloudStor web interface. If we want to download a zipped version of the child folder, we just add `download` to the url:

<https://cloudstor.aarnet.edu.au/plus/s/QOmnqpGQCNCSC2h/download?path=/angry-penguins-broadsheet-nla.obj-320790312>

This should download a file named `angry-penguins-broadsheet-nla.obj-320790312.zip`.

In [17]:
# Test link
response = requests.get(
    "https://cloudstor.aarnet.edu.au/plus/s/QOmnqpGQCNCSC2h/download?path=/angry-penguins-broadsheet-nla.obj-320790312"
)
assert response.status_code == 200

# The downloaded filename is in the response headers
downloaded_file = re.search(
    r'filename="(.+)"', response.headers["content-disposition"]
).group(1)

# Check that the downloaded filename is what we expect
assert downloaded_file == "angry-penguins-broadsheet-nla.obj-320790312.zip"

## Link to the child of a child

If we want to open a folder that is a child of a child we just extend the `path` value. This link goes to `angry-penguins-broadsheet-nla.obj-320790312/texts` in the parent folder:

<https://cloudstor.aarnet.edu.au/plus/s/QOmnqpGQCNCSC2h?path=/angry-penguins-broadsheet-nla.obj-320790312/texts>

In [20]:
# Test link
response = requests.get(
    "https://cloudstor.aarnet.edu.au/plus/s/QOmnqpGQCNCSC2h?path=/angry-penguins-broadsheet-nla.obj-320790312/texts"
)
assert response.status_code == 200

## Download a single file from a child folder

If we want to download a single file in a child folder we can just add the filename to the path, in this case `nla.obj-320790312-issues.csv`.

<https://cloudstor.aarnet.edu.au/plus/s/QOmnqpGQCNCSC2h/download?path=/angry-penguins-broadsheet-nla.obj-320790312/nla.obj-320790312-issues.csv>

Alternatively, we can add the `files` parameter and set it to the name of the file:

<https://cloudstor.aarnet.edu.au/plus/s/QOmnqpGQCNCSC2h/download?path=/angry-penguins-broadsheet-nla.obj-320790312&files=nla.obj-320790312-issues.csv>

In [21]:
# Test link using path parameter
response = requests.get(
    "https://cloudstor.aarnet.edu.au/plus/s/QOmnqpGQCNCSC2h/download?path=/angry-penguins-broadsheet-nla.obj-320790312/nla.obj-320790312-issues.csv"
)
assert response.status_code == 200

# The downloaded filename is in the response headers
downloaded_file = re.search(
    r'filename="(.+)"', response.headers["content-disposition"]
).group(1)

# Check that the downloaded filename is what we expect
assert downloaded_file == "nla.obj-320790312-issues.csv"

In [22]:
# Test link using files parameter
response = requests.get(
    "https://cloudstor.aarnet.edu.au/plus/s/QOmnqpGQCNCSC2h/download?path=/angry-penguins-broadsheet-nla.obj-320790312&files=nla.obj-320790312-issues.csv"
)
assert response.status_code == 200

# The downloaded filename is in the response headers
downloaded_file = re.search(
    r'filename="(.+)"', response.headers["content-disposition"]
).group(1)

# Check that the downloaded filename is what we expect
assert downloaded_file == "nla.obj-320790312-issues.csv"

## Download a selection of files from a child folder

If we want to download a selection of files from a folder rather than the whole folder, we can also use the `files` parameter. This link downloads two files from the `texts` folder, `angry-penguins-broadsheet-collection-no-1-nla.obj-320791009.txt` and `angry-penguins-broadsheet-collection-no-2-nla.obj-320791023.txt`, packaged as a single `zip` file. Note the use of `[]` with the `files` parameter to pass multiple values:

<https://cloudstor.aarnet.edu.au/plus/s/QOmnqpGQCNCSC2h/download?path=/angry-penguins-broadsheet-nla.obj-320790312/texts&files[]=angry-penguins-broadsheet-collection-no-1-nla.obj-320791009.txt&files[]=angry-penguins-broadsheet-collection-no-2-nla.obj-320791023.txt>

The files will be zipped up into a file with the name of their parent folder – so in this case the downloaded file will be named `texts.zip`.

In [26]:
# Test link using files parameter
response = requests.get(
    "https://cloudstor.aarnet.edu.au/plus/s/QOmnqpGQCNCSC2h/download?path=/angry-penguins-broadsheet-nla.obj-320790312/texts&files[]=angry-penguins-broadsheet-collection-no-1-nla.obj-320791009.txt&files[]=angry-penguins-broadsheet-collection-no-2-nla.obj-320791023.txt"
)
assert response.status_code == 200

# The downloaded filename is in the response headers
downloaded_file = re.search(
    r'filename="(.+)"', response.headers["content-disposition"]
).group(1)

# Check that the downloaded filename is what we expect
assert downloaded_file == "texts.zip"